In [239]:
import pickle
import pandas as pd
import numpy as np
from datetime import time

In [240]:
def hour_min(t):
    #Takes time in minutes and returns hours and minutes as int format
    h = int(t/60) 
    m = t%60
    return str(h)+":"+str(m)
print(hour_min(90))

1:30


In [241]:
#make a class of flight related data
class flight:
    'A class of flights with attributes flight no, orig station, origin_time, dest station, dest_time, duration'
    def __init__(self, num, aircraft, orig, dest, dep, arr):
        self.num = num
        self.aircraft = aircraft
        self.orig = orig
        self.dest = dest
        self.dep = dep
        self.arr = arr
        self.dur = arr.hour*60+arr.minute - (dep.hour*60+dep.minute)
        self.dep = self.dep.hour*60 + self.dep.minute
        if self.dur<0:
            self.dur+=1440
            self.arr = (self.arr.hour+24)*60 + self.arr.minute
        else:
            self.arr = self.arr.hour*60 + self.arr.minute
        
    def display(self):
        print("This is flight number-%d, aircraft=%s, Start-%s, End-%s, Dep-%s, arrival-%s, flight duration=%d minutes"%(self.num,  \
        self.aircraft, self.orig, self.dest, hour_min(self.dep), hour_min(self.arr), self.dur))

In [242]:
class maintenance:
    '''A class of maintenance with the attributes '''
    def __init__(self, ind, aircraft, airport, main_type, start, end):
        self.ind = ind
        self.aircraft = aircraft
        self.airport = airport
        self.main_type = main_type
        self.start = start.hour*60+start.minute
        self.end = end.hour*60+end.minute
        
        
    def display(self):
        print("This is maintenance number-%d, aircraft=%s airport-%s, type:%s, start-%d, end-%d"%(self.ind, self.aircraft,\
        self.airport, self.main_type, self.start,  self.end))

In [243]:
class NOTAM:
    '''A class of maintenance with the attributes '''
    def __init__(self, ind, airport, main_type, start, end):
        self.ind = ind
        self.airport = airport
        self.start = start.hour*60+start.minute
        self.end = end.hour*60+end.minute
        
        
    def display(self):
        print("This is NOTAM number-%d, airport-%s, start-%d, end-%d"%(self.ind, self.airport,\
                                                self.start,  self.end))

In [244]:
#make a class of aircraft related data
class aircraft:
    '''A class of aircrafts with the attributes aircraft number, aircraft type,
    aircraft start available airport, start available time'''
    def __init__(self, ind, model_type, start_airport, start_time):
        self.ind = ind
        self.model_type = model_type
        self.start_time = start_time
        self.start_airport = start_airport
        
    def display(self):
        print("This is aicraft number-%d, model type-%s, start-%d, start station-%s"%(self.ind, self.model_type,\
                                                self.start_time, self.start_airport))

In [245]:
#Make a class of scheduled maintenance 
class penalty:
    '''A class of penalty with the attributes'''
    def __init__(self, ind, flight, cancel_cost, per_hour_delay_cost):
        self.ind = ind
        self.flight = flight
        self.cancel_cost = cancel_cost
        self.per_hour_delay_cost = per_hour_delay_cost
        
    def display(self):
        print("This is penalty number-%d, flight-%s, cancel cost-%d, per hour delay cost-%d"%(self.ind, self.flight,\
                                                self.cancel_cost, self.per_hour_delay_cost))

In [246]:
#Reading the data of flight and saving as a dictionary of key=flight no and value
df_schedule = pd.read_excel("Input Files/Sample 8/Flight Schedule 4 Nov.xlsx")
print(len(df_schedule))
df_schedule = df_schedule[['TABBID','TAIL', 'DEP', 'ARR', 'RTDIST', 'RTAIST' ]]
#df_schedule = df_schedule[~df_schedule.RTD.str.contains("01-JUN")]
#df_schedule = df_schedule[~df_schedule.TAIL.str.contains("VTAIW")]
#df_schedule = df_schedule[~df_schedule.TAIL.str.contains("VTAIZ")]
flight_list = df_schedule.values.tolist()
print(len(flight_list))
for f in range(len(flight_list)):
    flight_list[f][4] = time(hour=int(flight_list[f][4][12:14]), minute = int(flight_list[f][4][15:]))
    flight_list[f][5] = time(hour=int(flight_list[f][5][12:14]), minute = int(flight_list[f][5][15:]))
flight_obj_dict = {}
tails = []
for f in flight_list:
    if f[0] in list(flight_obj_dict.keys()):
        print(f[0])
    flight_obj_dict[f[0]]=flight(*f)
    #Update tail list
    if f[1] not in tails:
        tails.append(f[1])
flight_obj_dict[flight_list[0][0]].display()
print("Length of dict keys:",len(list(flight_obj_dict.keys())))
with open("Preprocessed Objects/Sample 1/flight_dict", "wb") as file:
    pickle.dump(flight_obj_dict, file)

166
166
This is flight number-3496, aircraft=VTICD, Start-LKO, End-DEL, Dep-8:50, arrival-10:20, flight duration=90 minutes
Length of dict keys: 166


In [247]:
#flight_obj_dict[3638].display()

In [248]:
#flight_obj_dict[3514].display()

In [249]:
len(list(flight_obj_dict.keys()))

166

In [250]:
#Reading the data of maintenance and saving the objects in a dictionary
df_maint = pd.read_excel("Input Files/Sample 8/MaintenaneSchedule2June.xlsx")
df_maint=df_maint.dropna()
maint_list=df_maint.values.tolist()
maint_obj_dict = {}
for m in maint_list:
    maint_obj_dict[m[1]]=maintenance(*m)
#maint_obj_dict['VTEXJ'].display()
with open("Preprocessed Objects/Sample 1/maint_dict", "wb") as file:
    pickle.dump(maint_obj_dict, file)

In [251]:
#maint_obj_dict['VTEXJ'].start

In [252]:
#Reading the data of NOTAMs and saving the objects in a dictionary
df_NOTAM = pd.read_excel("Input Files/Sample 8/Notam_airportClosure.xlsx")
df_NOTAM = df_NOTAM.dropna()
notam_list = df_NOTAM.values.tolist()
notam_obj_dict = {}
for n in notam_list:
    notam_obj_dict[n[1]] = NOTAM(*n)
with open("Preprocessed Objects/Sample 1/notam_dict", "wb") as file:
    pickle.dump(notam_obj_dict, file)

In [253]:
#notam_obj_dict['LKO'].display()

In [254]:
#Reading the data of aircrafts and the starting available time
df_start = pd.read_excel("Input Files/Sample 8/StartAvailable.xlsx")
df_start = df_start.dropna()
start_list = df_start.values.tolist()
start_obj_dict = {}
for s in start_list:
    start_obj_dict[s[1]] = aircraft(*s)

#First Flight of Aircraft c: Fc[c]
Fc={}
for c in tails:
    Fsc=[]
    for f in flight_obj_dict.keys():
        if flight_obj_dict[f].aircraft == c:
            Fsc.append(f)
    Fc[c] = min(Fsc,key = lambda f:df_schedule.loc[df_schedule.TABBID==f, 'RTDIST'].values)
ct = len(list(start_obj_dict.keys()))
#Update dictionary of start time of every aircraft
for c in tails:    
    if c not in list(start_obj_dict.keys()):
        start_obj_dict[c] = aircraft(ct, c, flight_obj_dict[Fc[c]].orig, flight_obj_dict[Fc[c]].dep)
#start_obj_dict['VTEXJ'].display()
with open("Preprocessed Objects/Sample 1/start_dict", "wb") as file:
    pickle.dump(start_obj_dict, file)

In [255]:
#Reading the data of penalties
df_pen = pd.read_excel("Input Files/Sample 8/Penalties.xlsx")
df_pen = df_pen.dropna()
pen_list = df_pen.values.tolist()
pen_obj_dict = {}
for p in pen_list:
    pen_obj_dict[p[0]] = penalty(*p)
with open("Preprocessed Objects/Sample 1/penalty_dict", "wb") as file:
    pickle.dump(pen_obj_dict, file)

In [256]:
#-----Make a dictionary of starting flights for each airport
airport=[]
for i in flight_obj_dict.keys():
    if flight_obj_dict[i].orig not in airport:
        airport.append(flight_obj_dict[i].orig)
    if flight_obj_dict[i].dest not in airport:
        airport.append(flight_obj_dict[i].dest)
startflights = {a:[] for a in airport}
for f in flight_obj_dict.values():
    if f.orig in list(startflights.keys()):
        startflights[f.orig].append(f)
    else:
        startflights[f.orig] = [f]
print(len(list(startflights.keys())))

46


In [257]:
#make a class of LOF related data
class lof:
    'A class of route with attributes flights and available time after the end flight'
    def __init__(self, flights, avail_time_end, port_end, cost, inf):
        self.flights= flights
        self.avail_time_end=avail_time_end
        self.port_end=port_end
        self.cost=cost
        self.info = inf
        
    def extend(self, F_added):
        L=[]            #defining a list of LOFs
        for f in F_added:
            l=lof(self.flights+[f[0]], f[1], f[2])
            L.append(l)
        return L
    
    def display(self):
        print(self.info)
        '''
        print("LOF")
        for f in self.flights:
            print(" %d"%f.num , end=" ")
        print("\n avail time in hours= %f"%(self.avail_time_end/60))
        print("End port="+str(self.port_end))
        print("Cost=$"+str(self.cost))
        '''
        
    def is_f(self, f):
        for i in self.flights:
            if i!='m' and f.num == i.num:
                return 1
        return 0
    
    def is_p(self, p):
        if p==self.port_end:
            return 1
        else:
            return 0

In [258]:

#Dictionary of flights of every aircraft
flights = {}
for c in start_obj_dict.keys():
    Lc=[]
    for f in flight_obj_dict.keys():
        if flight_obj_dict[f].aircraft == c:
            Lc.append(f)
    Lc.sort(key = lambda f:df_schedule.loc[df_schedule.TABBID==f, 'RTDIST'].values)
    flights[c]=Lc
print(flights)

{'VTICD': [3496, 3497, 3498, 3499, 3500], 'VTICX': [3628, 3629], 'VTICF': [3746, 3747], 'VTICG': [3875, 3876, 3877], 'VTICH': [4008, 4009, 4010], 'VTICM': [4140, 4141, 4142, 4143], 'VTICN': [4275], 'VTICO': [4401, 4402], 'VTICB': [4524], 'VTICQ': [4643, 4644, 4645], 'VTXDX': [4819, 4820, 4821, 4822, 4823], 'VTXES': [4904, 4905, 4906, 4907], 'VTXEP': [5007, 5008, 5009], 'VTXEI': [5140, 5141, 5142, 5143, 5144], 'VTXED': [5248, 5249, 5250, 5251, 5252, 5253], 'VTXEX': [5343, 5344, 5345, 5346, 5347], 'VTXEF': [5465, 5466, 5467, 5468], 'VTXEG': [5589, 5590, 5591], 'VTXEL': [6200, 6201, 6202, 6203], 'VTXEM': [6305, 6306], 'VTXEN': [6415, 6416, 6417, 6418, 6419], 'VTXEO': [6531, 6532, 6533], 'VTXEQ': [6750, 6751, 6752], 'VTXER': [6865, 6866], 'VTXEA': [6992, 6993, 6994, 6995, 6996], 'VTXET': [7134, 7135, 7136, 7137, 7138], 'VTXEU': [7248, 7249, 7250, 7251], 'VTBBP': [8082], 'VTBBD': [8179, 8180, 8181, 8182], 'VTBBX': [8258, 8259], 'VTBBJ': [8346, 8347, 8348, 8349], 'VTBBK': [8429, 8430, 8431, 

In [259]:
#Run this cell to store initial line of flights for every aircraft and its cost
Pos_lof_c = {}
mark_c = [] # mark aircraft with recovery by delay not possible
for c in start_obj_dict.keys():
    #if c!='VTAYA':
    first_avail_t = start_obj_dict[c].start_time + flight_obj_dict[flights[c][0]].dur + 30
    if first_avail_t-flight_obj_dict[flights[c][0]].arr-30>0:
        l = lof([flight_obj_dict[flights[c][0]]], first_avail_t, flight_obj_dict[flights[c][0]].dest, \
                20*(first_avail_t-flight_obj_dict[flights[c][0]].arr-30),\
               str(flights[c][0])+" "+ flight_obj_dict[flights[c][0]].orig+" "+flight_obj_dict[flights[c][0]].dest+ " "\
                +hour_min(first_avail_t - flight_obj_dict[flights[c][0]].dur - 30)+" "+\
                hour_min(first_avail_t-30))
    else:
        l = lof([flight_obj_dict[flights[c][0]]], flight_obj_dict[flights[c][0]].arr+30,\
                flight_obj_dict[flights[c][0]].dest, 0,\
               str(flights[c][0])+" "+flight_obj_dict[flights[c][0]].orig+" "+\
                flight_obj_dict[flights[c][0]].dest+ " "+ hour_min(flight_obj_dict[flights[c][0]].dep)+" "+\
                hour_min(flight_obj_dict[flights[c][0]].arr))
    for f in flights[c][1:]:
        if c in list(maint_obj_dict.keys()) and l.avail_time_end>maint_obj_dict[c].start and 'm' not in l.flights:
            print("-#--#-#-#-#-#-Updating l-#-#-#-#-#")
            l = lof(l.flights+['m'], l.avail_time_end + maint_obj_dict[c].end-maint_obj_dict[c].start,l.port_end, l.cost,\
                       l.info+"\n Maintenance:"+str(maint_obj_dict[c].end-maint_obj_dict[c].start)+" min")
        max_time=l.avail_time_end + flight_obj_dict[f].dur + 30 if l.avail_time_end>flight_obj_dict[f].dep else flight_obj_dict[f].arr+30
        if l.port_end in list(notam_obj_dict.keys()):
            if max_time-(flight_obj_dict[f].dur + 30)>=notam_obj_dict[l.port_end].start and max_time-(flight_obj_dict[f].dur + 30)<=notam_obj_dict[l.port_end].end:
                max_time = notam_obj_dict[l.port_end].end + flight_obj_dict[f].dur + 30
        if flight_obj_dict[f].dest in list(notam_obj_dict.keys()):
            if l.avail_time_end>flight_obj_dict[f].dep:
                if l.avail_time_end+flight_obj_dict[f].dur>=notam_obj_dict[flight_obj_dict[f].dest].start and l.avail_time_end+flight_obj_dict[f].dur<=notam_obj_dict[flight_obj_dict[f].dest].end:
                    t_av = notam_obj_dict[flight_obj_dict[f].dest].end + 30
                    if t_av>max_time:
                        max_time=t_av
            else:

                if flight_obj_dict[f].arr.hour>=notam_obj_dict[flight_obj_dict[f].dest].start and flight_obj_dict[f].arr.hour<=notam_obj_dict[flight_obj_dict[f].dest].end:
                    t_av = notam_obj_dict[flight_obj_dict[f].dest].end + 30
                    if t_av>max_time:
                        max_time=t_av
        if max_time-30>27*60:
            mark_c.append(c)
            break
        l=lof(l.flights+[flight_obj_dict[f]], max_time, flight_obj_dict[f].dest,\
              l.cost + 20*(max_time-flight_obj_dict[f].arr-30),\
               l.info + "\n"+str(f)+" "+flight_obj_dict[f].orig+" "+\
              flight_obj_dict[f].dest+" "+hour_min(max_time - flight_obj_dict[f].dur - 30)+" "+\
                hour_min(max_time-30))
    if c not in mark_c:
        Pos_lof_c[c]=[l]
print(Pos_lof_c)
print(mark_c)

{'VTICD': [<__main__.lof object at 0x000001BB2AF7FB70>], 'VTICX': [<__main__.lof object at 0x000001BB2AF7F908>], 'VTICF': [<__main__.lof object at 0x000001BB2B2BD550>], 'VTICG': [<__main__.lof object at 0x000001BB2AF7F588>], 'VTICH': [<__main__.lof object at 0x000001BB2AF7FE80>], 'VTICM': [<__main__.lof object at 0x000001BB2AE0B9B0>], 'VTICN': [<__main__.lof object at 0x000001BB2B2BD710>], 'VTICO': [<__main__.lof object at 0x000001BB2AE0BEB8>], 'VTICB': [<__main__.lof object at 0x000001BB2B2BD518>], 'VTICQ': [<__main__.lof object at 0x000001BB2AE0BC18>], 'VTXDX': [<__main__.lof object at 0x000001BB2AE0B7B8>], 'VTXES': [<__main__.lof object at 0x000001BB2AD4B320>], 'VTXEP': [<__main__.lof object at 0x000001BB2A8A3F98>], 'VTXEI': [<__main__.lof object at 0x000001BB2AE0B978>], 'VTXED': [<__main__.lof object at 0x000001BB2AD4B128>], 'VTXEX': [<__main__.lof object at 0x000001BB2AD4B198>], 'VTXEF': [<__main__.lof object at 0x000001BB2AD4B048>], 'VTXEG': [<__main__.lof object at 0x000001BB2AD

In [260]:
Pos_lof_c['VTICX'][0].display()

3628 MAA HYD 13:55 14:55
3629 HYD MAA 15:55 17:0


In [261]:
mark_c

[]

In [262]:
#Pos_lof_c['VTAYA']  = [lof([], 720, 'MAA', 28800*4)]
for c in mark_c:
    Pos_lof_c[c] = [lof([], start_obj_dict[c].start_time, start_obj_dict[c].start_airport, 28800*4, " ")]

In [263]:
for c in start_obj_dict.keys():
    print(c,"=",Pos_lof_c[c][0].cost, end=" / ")

VTICD = 9400 / VTICX = 0 / VTICF = 0 / VTICG = 0 / VTICH = 0 / VTICM = 0 / VTICN = 0 / VTICO = 0 / VTICB = 0 / VTICQ = 0 / VTXDX = 0 / VTXES = 0 / VTXEP = 0 / VTXEI = 0 / VTXED = 0 / VTXEX = 0 / VTXEF = 0 / VTXEG = 0 / VTXEL = 0 / VTXEM = 0 / VTXEN = 0 / VTXEO = 0 / VTXEQ = 0 / VTXER = 0 / VTXEA = 0 / VTXET = 0 / VTXEU = 0 / VTBBP = 0 / VTBBD = 0 / VTBBX = 0 / VTBBJ = 0 / VTBBK = 0 / VTBBL = 0 / VTBBM = 0 / VTBBQ = 0 / VTBBT = 300 / VTBBV = 0 / VTBBW = 0 / VTBBE = 0 / VTAIP = 0 / VTAII = 0 / VTAIF = 0 / VTAIG = 0 / VTAIH = 0 / VTAIN = 0 / VTAIO = 0 / VTAIB = 0 / VTAIR = 0 / VTAIA = 0 / VTAIV = 0 / 

In [264]:
Pos_lof_c['VTICD'][0].display()
with open("Preprocessed Objects/Sample 1/initial_lof_dict", "wb") as file:
    pickle.dump(Pos_lof_c, file)

3496 LKO DEL 13:15 14:45
3497 DEL GOI 15:15 17:20
3498 GOI DEL 17:50 20:25
3499 DEL ATQ 20:55 22:10
3500 ATQ DEL 22:45 24:0


In [82]:
#Checks extensions of LOF conditions
def check_conditions(f, airplane_avail_time, avail_port):
    f1=f2=f3=f4=0
    f2=1
    #Condition 1 : Night curfew
    if f.dur+airplane_avail_time<24*60:
        f1=1
   
    if f1==1:
        return 1
    else:
        return 0

def create_next_l(l , f, notam_obj_dict, mode):
    #Create next level lof by adding f in the lof l. Two modes of the function 
    if mode==1:
        max_time = 0
        if l.port_end in list(notam_obj_dict.keys()):
            if l.avail_time_end>=notam_obj_dict[l.port_end].start and l.avail_time_end<=notam_obj_dict[l.port_end].end:
                max_time = notam_obj_dict[l.port_end].end + f.dur + 30
        if f.dest in list(notam_obj_dict.keys()):
            if l.avail_time_end+f.dur>=notam_obj_dict[f.dest].start and l.avail_time_end+f.dur<=notam_obj_dict[f.dest].end:
                t_av = notam_obj_dict[f.dest].end + 30
                if t_av>max_time:
                    max_time=t_av
        if l.port_end not in list(notam_obj_dict.keys()) and f.dest not in list(notam_obj_dict.keys()):
            max_time = l.avail_time_end+f.dur + 30 
        new_l=lof(l.flights+[f], max_time, f.dest, l.cost + 20*(max_time-f.arr.hour*60-f.arr.minute-30))         
    elif mode==2:
        max_time = 0
        if f.orig in list(notam_obj_dict.keys()):
            if f.dep.hour*60+f.dep.minute>=notam_obj_dict[f.orig].start and f.dep.hour*60+f.dep.minute<=notam_obj_dict[f.orig].end:
                max_time = notam_obj_dict[f.orig].end + f.dur + 30
        if f.dest in list(notam_obj_dict.keys()):
            if f.dep.hour*60+f.dep.minute+f.dur>=notam_obj_dict[f.dest].start and f.dep.hour*60+f.dep.minute+f.dur<=notam_obj_dict[f.dest].end:
                t_av = notam_obj_dict[f.dest].end + 30
                if t_av>max_time:
                    max_time=t_av
        if f.orig not in list(notam_obj_dict.keys()) and f.dest not in list(notam_obj_dict.keys()):
            max_time = l.avail_time_end+f.dur + 30 
        new_l=lof(l.flights+[f], max_time, f.dest, l.cost + 20*(max_time-f.arr.hour*60-f.arr.minute-30))      
    if max_time>24*60:
        return -1
    return new_l

In [23]:
#creates a start list of lofs for a particular aircraft given its start time and start airport+
def add_s(startflights, airplane_avail_time, avail_port): 
    L=[]
    for f in startflights[avail_port]:
        if  airplane_avail_time.hour*60+airplane_avail_time.minute-(f.dep.hour*60+f.dep.minute)>=0:
            if check_conditions(f, airplane_avail_time, avail_port):
                L.append(lof([f], f.dur+airplane_avail_time.hour*60+airplane_avail_time.minute+30, f.dest, 20*(airplane_avail_time.hour*60+airplane_avail_time.minute-(f.dep.hour*60+f.dep.minute))))
        else:
            if check_conditions(f, airplane_avail_time, avail_port):
                L.append(lof([f], f.dur+f.dep.hour*60+f.dep.minute+30, f.dest, 0))
    return L

In [24]:
L=add_s(startflights, flight_obj_dict[7766].dep, flight_obj_dict[7766].orig)
print(L)
L[0].flights[0].display()
L[2].flights[0].display()
L[3].flights[0].display()
print(L[3].cost)

KeyError: 7766

In [ ]:
# grows next layer into the list of LOF
def add_nex(L, startflights, notam_obj_dict):                
    next_L=L[:]
    flag=0
    for l in L:
        flag_l=0
        for f in startflights[l.port_end]:
            if f.dur+l.avail_time_end<23*60 and l.avail_time_end-(f.dep.hour*60+f.dep.minute)>=0 and l.is_f(f)==0:
                if create_next_l(l , f, notam_obj_dict, 1)!=-1:
                    flag_l=1
                    flag=1
                    new_l = create_next_l(l , f, notam_obj_dict, 1)
                    next_L.append(new_l)
            elif f.dur+f.dep.hour*60+f.dep.minute<23*60 and l.avail_time_end-(f.dep.hour*60+f.dep.minute)<=0 and l.is_f(f)==0:
                if create_next_l(l , f, notam_obj_dict, 1)!=-1:
                    flag_l=1
                    flag=1
                    new_l = create_next_l(l , f, notam_obj_dict, 2)
                    next_L.append(new_l)
        if flag_l==1:
            next_L.remove(l)
    next_L[-1].display()
    L[:]=next_L
    L[-1].display()
    return flag

In [ ]:
add_nex(L, startflights, notam_obj_dict)

In [45]:
t1 = time(hour=3, minute=30)
s1 = t1.strftime("%H hours %M minutes")
print(s1)

03 hours 30 minutes


In [ ]:
Pos_lof_c={i:[] for i in list(start_obj_dict.keys())}
for i in Pos_lof_c.keys():
    Pos_lof_c[i]=add_s(startflights, start_obj_dict[i].start_time, start_obj_dict[i].start_airport)
    flag=1
    while flag==1:
        flag=add_nex(Pos_lof_c[i], startflights, notam_obj_dict)

In [88]:
start_obj_dict['VTCIG'].display()
print(notam_obj_dict.keys())
notam_obj_dict['VGA'].display()

This is aicraft number-2, model type-VTCIG, start-360, start station-BLR
dict_keys(['LKO', 'VGA', 'BKK', 'FRA'])
This is NOTAM number-2, airport-VGA, start-660, end-960


False